## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-14-40-06 +0000,nyt,"Patrick Ryan, ‘Terror Priest’ Who Aided the I....",https://www.nytimes.com/2025/07/25/world/europ...
1,2025-07-26-14-31-11 +0000,startribune,New details about police activity after lawmak...,https://www.startribune.com/new-details-from-c...
2,2025-07-26-14-21-50 +0000,startribune,Charge dismissed against woman in Minneapolis ...,https://www.startribune.com/charge-dismissed-a...
3,2025-07-26-14-00-54 +0000,nyt,"In Russia, Corruption Cases Follow Battlefield...",https://www.nytimes.com/2025/07/26/world/europ...
4,2025-07-26-14-00-00 +0000,wsj,A Cosmic Mystery: Is China Building the World’...,https://www.wsj.com/world/china/a-cosmic-myste...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2586/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,49
6,new,19
117,gaza,13
356,he,10
289,out,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
241,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,97
223,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...,97
56,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...,89
199,2025-07-25-18-31-45 +0000,nypost,Elon Musk hits back at Trump after president s...,https://nypost.com/2025/07/25/business/elon-mu...,87
128,2025-07-25-23-06-53 +0000,wapo,"Amid starvation scenes in Gaza, Trump administ...",https://www.washingtonpost.com/national-securi...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
241,97,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo
167,63,2025-07-25-20-45-00 +0000,wsj,Rep. Alexandria Ocasio-Cortez of New York stil...,https://www.wsj.com/politics/policy/aoc-met-ga...
223,48,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...
10,46,2025-07-26-13-24-33 +0000,nypost,Thailand and Cambodia trade accusations as dea...,https://nypost.com/2025/07/26/world-news/thail...
82,35,2025-07-26-04-59-57 +0000,nypost,"Australian train passenger, 74, arrested for t...",https://nypost.com/2025/07/26/world-news/austr...
126,33,2025-07-25-23-24-00 +0000,wsj,HHS Secretary RFK Jr. plans to remove all memb...,https://www.wsj.com/health/healthcare/rfk-heal...
109,31,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
202,31,2025-07-25-18-18-54 +0000,wapo,Russia rules out Putin-Zelensky meeting until ...,https://www.washingtonpost.com/world/2025/07/2...
138,28,2025-07-25-22-10-39 +0000,bbc,"'I witnessed war crimes' in Gaza, former worke...",https://www.bbc.com/news/videos/cy8k8045nx9o
56,27,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
